# Transfer data between MongoDB and SQL database. Map-reduce in MongoDB

**Tasks to complete:**  
* Import "reviews_electronics.16.json" file into MongoDB.  

* Create a SQL database and table and import each document from the collection into the SQL table.

* Map reduce to create collection "avg_scores" by product.  

* Map reduce to create collection "weighted_avg_score" by product.

## Import data into MongoDB
Create a collection called “electronics” in “amazon” that contains each line of “reviews_electronics.16.json” as an individual document.

In [1]:
import pandas as pd
from pymongo import MongoClient
import json
from bson.code import Code
import pymysql
from tqdm import tqdm
import re
from bs4 import BeautifulSoup as bs 
from datetime import datetime

In [2]:
# Connect to the MongoDB and create collection "electronics" in "amazon" database.
client = MongoClient('localhost', 27017)
db = client['amazon']
coll = db['electronics']

In [3]:
data = []
with open('reviews_electronics.16.json', encoding="UTF-8") as json_f:
    for line in json_f:
        data.append(json.loads(line))
    coll.insert_many(data)

In [4]:
# check if the database exists
dblist = client.list_database_names()
if "amazon" in dblist:
    print("The database exists.")

The database exists.


Write code that reads all documents in “electronics”, and prints the first 25 documents to screen.

In [5]:
results = coll.find()
for result in results[:25]:
     print(result)

{'_id': ObjectId('5e5ca1dfb2b52b52e00371b6'), 'reviewerID': 'AKM1MP6P0OYPR', 'asin': '0132793040', 'reviewerName': 'Vicki Gibson "momo4"', 'helpful': [1, 1], 'reviewText': 'Corey Barker does a great job of explaining Blend Modes in this DVD. All of the Kelby training videos are great but pricey to buy individually. If you really want bang for your buck just subscribe to Kelby Training online.', 'overall': 5.0, 'summary': 'Very thorough', 'unixReviewTime': 1365811200, 'reviewTime': '04 13, 2013'}
{'_id': ObjectId('5e5ca1dfb2b52b52e00371b7'), 'reviewerID': 'A2X8VX4DPMQFQQ', 'asin': 'B00E4KP4W6', 'reviewerName': 'lily68', 'helpful': [1, 1], 'reviewText': "I can't believe I waited to long to switch to a glass screen protector.  I love this.  It feels and looks like there is no protector on.  It does show fingerprints, which I think is inevitable unless you use a matte finish screen protector, but they wipe right away. I would definitely recommend this! Easier to apply than the films too!",

## Create a SQL database and table to hold each document from “electronics”. 

In [5]:
pw = "" # a dynamic password for users to enter

In [6]:
# Create eBay Database and eBay_items table.
conn = pymysql.connect(host='localhost', user = 'root', password = pw)
cursor = conn.cursor()

In [7]:
# Create database amazon_SQL
SQL_DB = "amazon_SQL"
cursor.execute('DROP DATABASE IF EXISTS '+SQL_DB)
query_DB = "CREATE DATABASE IF NOT EXISTS " + SQL_DB + ";"
cursor.execute(query_DB)

/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1008, "Can't drop database 'amazon_sql'; database doesn't exist")
  result = self._query(query)


1

In [9]:
# Create an empty table called electronics_SQL.
SQL_table = "electronics_SQL"
cursor.execute('DROP TABLE IF EXISTS ' + SQL_DB + '.' + SQL_table)
query_TB = "CREATE TABLE IF NOT EXISTS " + SQL_DB + "." + SQL_table + "(" \
+ "id VARCHAR(100) NOT NULL PRIMARY KEY" \
+ ", reviewerID VARCHAR(50)" \
+ ", asin VARCHAR(50)" \
+ ", reviewerName VARCHAR(100)" \
+ ", helpful VARCHAR(1000)" \
+ ", overall int(2)" \
+ ", summary VARCHAR(3000)" \
+ ", unixReviewTime int(15)" \
+ ", ReviewTime date" \
+ ");"

cursor.execute(query_TB);
cursor.close()
conn.close()

/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1681, 'Integer display width is deprecated and will be removed in a future release.')
  result = self._query(query)


### (1) reads all documents from “electronics” in MongoDB

In [6]:
results = coll.find()
# list to convert the cursor type.
results_list = list(results)

In [7]:
results_list

[{'_id': ObjectId('5e5ca1dfb2b52b52e00371b6'),
  'reviewerID': 'AKM1MP6P0OYPR',
  'asin': '0132793040',
  'reviewerName': 'Vicki Gibson "momo4"',
  'helpful': [1, 1],
  'reviewText': 'Corey Barker does a great job of explaining Blend Modes in this DVD. All of the Kelby training videos are great but pricey to buy individually. If you really want bang for your buck just subscribe to Kelby Training online.',
  'overall': 5.0,
  'summary': 'Very thorough',
  'unixReviewTime': 1365811200,
  'reviewTime': '04 13, 2013'},
 {'_id': ObjectId('5e5ca1dfb2b52b52e00371b7'),
  'reviewerID': 'A2X8VX4DPMQFQQ',
  'asin': 'B00E4KP4W6',
  'reviewerName': 'lily68',
  'helpful': [1, 1],
  'reviewText': "I can't believe I waited to long to switch to a glass screen protector.  I love this.  It feels and looks like there is no protector on.  It does show fingerprints, which I think is inevitable unless you use a matte finish screen protector, but they wipe right away. I would definitely recommend this! Easier

### (2) writes that content to “electronics_SQL” in SQL. Print the first 25 documents to the screen. 

In [ ]:
# for el in result_list:
#     # deal with the objectid, or you can create the primary key yourself by using incremented.
    
#     # format the reviewTime
#     el["reviewTime"] = datetime.strptime(el["reviewTime"],'%m %d, %Y').date()
#     el["reviewTime"] = datetime.strftime(el["reviewTime"],'%Y-%m-%d')
    
#     # convert helpful to string
#     el["helpful"] = str(el["helpful"])
    
#     # deal with missing reviewerName
#     if "reviewerName" not in el.keys():
#         el["reviewerName"] = None
        
# val_to_insert = result_list

In [57]:
results_df = pd.DataFrame(results_list)

# clean the id column, which is bson id type.
results_df["_id"] = results_df['_id'].astype(str)
results_df['_id'].replace('("ObjectId")','',regex=True, inplace = True)

# drop the review text column
results_df.drop(["reviewText"], axis = 1, inplace=True)

# format the reviewTime
results_df["reviewTime"] = pd.to_datetime(results_df["reviewTime"]).dt.date.astype(str)

# change the helpful format to string
results_df["helpful"] = results_df["helpful"].astype(str)

# change the nan values in dataframe to be None so as to be detected by MySQL
results_df = results_df.where(pd.notnull(results_df), None)

val_to_insert = results_df.values.tolist()

In [58]:
conn = pymysql.connect(host='localhost', user = 'root', password = pw, database=SQL_DB)
cursor = conn.cursor()

sql = "INSERT INTO "+ SQL_table + ("(id, asin,helpful,overall,reviewTime,reviewerID,"
                                   "reviewerName,summary,unixReviewTime)"
                                   " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)")

try:
    cursor.executemany(sql, val_to_insert)
    conn.commit()
    
except pymysql.IntegrityError:
    print('already exists in the database.')

conn.close()

In [61]:
conn = pymysql.connect(host='localhost', user = 'root', password = pw, database=SQL_DB)

SQL_Query = pd.read_sql_query(
'''
SELECT * FROM electronics_SQL
LIMIT 25;
''', conn)

SQL_Query

conn.close()

,id,reviewerID,asin,reviewerName,helpful,overall,summary,unixReviewTime,ReviewTime
0,5e5ca1dfb2b52b52e00371b6,AKM1MP6P0OYPR,0132793040,"Vicki Gibson ""momo4""","[1, 1]",5,Very thorough,1365811200,2013-04-13
1,5e5ca1dfb2b52b52e00371b7,A2X8VX4DPMQFQQ,B00E4KP4W6,lily68,"[1, 1]",5,LOVE this screen protector!!,1393459200,2014-02-27
2,5e5ca1dfb2b52b52e00371b8,A1JEKX4D58576X,B00E4KP4W6,Mara Milagros,"[2, 2]",5,excellent,1394496000,2014-03-11
3,5e5ca1dfb2b52b52e00371b9,ATM6RRB90D8DV,B00E4KP4W6,"Matthew D. Irmen ""Jazz Fan""","[1, 2]",4,"Nice looking, easy to install glass screen pro...",1393891200,2014-03-04
4,5e5ca1dfb2b52b52e00371ba,A3QOYJ97A6OMX7,B00E4KP4W6,pharmdaddyd,"[1, 1]",5,great for the price,1388361600,2013-12-30
5,5e5ca1dfb2b52b52e00371bb,AWO84NZ6VDYR2,B00E4KP4W6,shalane rogillio,"[0, 0]",1,"Not happy with this, would not recommend",1404777600,2014-07-08
6,5e5ca1dfb2b52b52e00371bc,A15ZEBTKX72NPB,B00E4KP4W6,"S. Marcus ""pdtpoet""","[1, 2]",5,Far better than film screens for iPhone or iPad!,1392249600,2014-02-13
7,5e5ca1dfb2b52b52e00371bd,A2ZI014DO0LWEW,B00E4KP4W6,SueinCa,"[0, 0]",5,Right On,1397174400,2014-04-11
8,5e5ca1dfb2b52b52e00371be,A3IN9CE5GG8VKR,B00E4KP4W6,Tess Brown,"[0, 0]",5,Tempered Glass Screen Protector,1399593600,2014-05-09
9,5e5ca1dfb2b52b52e00371bf,A1U9ZR7HQ5EZ6L,B00E4KP4W6,Thinker623,"[1, 1]",5,Great glass cover for IPad.,1391126400,2014-01-31


## Map Reduce to create collection "avg_scores" by product.

In [8]:
mapper_1 = Code("""
                    function () { emit(this.asin, this.overall)}
                """)


reducer_1 = Code("""
                   function(key, value) { return Array.avg(value) }
                 """)
query_1 = {"overall":{'$exists':1}}

In [9]:
avg_review = coll.map_reduce(mapper_1, reducer_1, "avg_scores", query=query_1)

In [10]:
# first 100 entries of avg_reviews
avg_review_df = pd.DataFrame(list(avg_review.find().limit(100)))
avg_review_df

,_id,value
0,0132793040,5.000000
1,B00E4KP4W6,4.545455
2,B00E4KP8VI,5.000000
3,B00E4KPMC8,2.000000
4,B00E4KQ5C4,5.000000
5,B00E4KQ9GG,3.285714
6,B00E4KQ9K2,5.000000
7,B00E4KQD4E,4.000000
8,B00E4KZBX8,4.000000
9,B00E4KZDJ0,5.000000


In [11]:
for avg_score in avg_review.find().limit(100):
    print(avg_score)

{'_id': '0132793040', 'value': 5.0}
{'_id': 'B00E4KP4W6', 'value': 4.545454545454546}
{'_id': 'B00E4KP8VI', 'value': 5.0}
{'_id': 'B00E4KPMC8', 'value': 2.0}
{'_id': 'B00E4KQ5C4', 'value': 5.0}
{'_id': 'B00E4KQ9GG', 'value': 3.2857142857142856}
{'_id': 'B00E4KQ9K2', 'value': 5.0}
{'_id': 'B00E4KQD4E', 'value': 4.0}
{'_id': 'B00E4KZBX8', 'value': 4.0}
{'_id': 'B00E4KZDJ0', 'value': 5.0}
{'_id': 'B00E4L35DA', 'value': 4.0}
{'_id': 'B00E4L3N9Q', 'value': 4.0}
{'_id': 'B00E4L48EA', 'value': 5.0}
{'_id': 'B00E4L7FLI', 'value': 1.0}
{'_id': 'B00E4L7TS2', 'value': 4.0}
{'_id': 'B00E4LAL82', 'value': 3.0}
{'_id': 'B00E4LBZZK', 'value': 5.0}
{'_id': 'B00E4LF2Z4', 'value': 4.333333333333333}
{'_id': 'B00E4LFP0G', 'value': 4.444444444444445}
{'_id': 'B00E4LFWWW', 'value': 4.4}
{'_id': 'B00E4LGTVU', 'value': 4.195658625514055}
{'_id': 'B00E4LGTXS', 'value': 3.923076923076923}
{'_id': 'B00E4LGVYA', 'value': 3.272727272727273}
{'_id': 'B00E4LGWLW', 'value': 3.5}
{'_id': 'B00E4LGXL6', 'value': 5.0}
{

## Map Reduce to create collection "weighted_avg_scores" by product.

In [12]:
mapper_2 = Code("""
                    function () { 
                        var key = this.asin;
                        var values = {wtd_scores: (this.helpful[0]+1)*this.overall, wt:(this.helpful[0]+1)};
                        emit(key, values);
                    }
                """)

reducer_2 = Code("""
                    function(key, value) {
                        var tot = {wtd_scores:0, wt:0};
                        for (var idx = 0; idx<value.length; idx++){
                             tot.wtd_scores += value[idx].wtd_scores;
                             tot.wt += value[idx].wt;
                        };
                    return tot; 
                    }
                 """)

finalizer = Code("""
                    function (key, tot){
                    wtd_avg = tot.wtd_scores/tot.wt;
                    return wtd_avg; 
                    }
                 """)
query_2 = {"overall": {'$exists':1}}

In [13]:
wtd_review = coll.map_reduce(map=mapper_2, reduce=reducer_2, out="weighted_avg", query=query_2, finalize=finalizer)

In [14]:
# first 100 entries of avg_reviews
wtd_review_df = pd.DataFrame(list(wtd_review.find().limit(100)))
wtd_review_df

,_id,value
0,0132793040,5.000000
1,B00E4KP4W6,4.684211
2,B00E4KP8VI,5.000000
3,B00E4KPMC8,2.000000
4,B00E4KQ5C4,5.000000
5,B00E4KQ9GG,3.687500
6,B00E4KQ9K2,5.000000
7,B00E4KQD4E,4.000000
8,B00E4KZBX8,4.000000
9,B00E4KZDJ0,5.000000


In [15]:
for wtd_score in wtd_review.find().limit(100):
    print(wtd_score)

{'_id': '0132793040', 'value': 5.0}
{'_id': 'B00E4KP4W6', 'value': 4.684210526315789}
{'_id': 'B00E4KP8VI', 'value': 5.0}
{'_id': 'B00E4KPMC8', 'value': 2.0}
{'_id': 'B00E4KQ5C4', 'value': 5.0}
{'_id': 'B00E4KQ9GG', 'value': 3.6875}
{'_id': 'B00E4KQ9K2', 'value': 5.0}
{'_id': 'B00E4KQD4E', 'value': 4.0}
{'_id': 'B00E4KZBX8', 'value': 4.0}
{'_id': 'B00E4KZDJ0', 'value': 5.0}
{'_id': 'B00E4L35DA', 'value': 3.0}
{'_id': 'B00E4L3N9Q', 'value': 4.0}
{'_id': 'B00E4L48EA', 'value': 5.0}
{'_id': 'B00E4L7FLI', 'value': 1.0}
{'_id': 'B00E4L7TS2', 'value': 4.0}
{'_id': 'B00E4LAL82', 'value': 3.0}
{'_id': 'B00E4LBZZK', 'value': 5.0}
{'_id': 'B00E4LF2Z4', 'value': 4.153846153846154}
{'_id': 'B00E4LFP0G', 'value': 4.434782608695652}
{'_id': 'B00E4LFWWW', 'value': 4.4}
{'_id': 'B00E4LGTVU', 'value': 3.9064516129032256}
{'_id': 'B00E4LGTXS', 'value': 4.0}
{'_id': 'B00E4LGVYA', 'value': 3.8}
{'_id': 'B00E4LGWLW', 'value': 3.8181818181818183}
{'_id': 'B00E4LGXL6', 'value': 5.0}
{'_id': 'B00E4LGY88', 'va